In [1]:
pnt_file = "/home/benr/wqmodels/ssm/wqm-test/inputs/ssm_pnt_wq.dat"
out_file = "data/ssm_pnt_2014.nc"

import numpy as np
from netCDF4 import Dataset

In [2]:
with open(pnt_file) as f:
    # The parsing logic here is is derived from the linkage instructions for the
    # model and direct examination of the source code
    
    # The first line does not contain important information and is treated only like
    # a filetype magic
    next(f)

    # The total number of discharge nodes
    num_qs = int(next(f))
    # All the node numbers with discharges
    nodes = np.loadtxt([next(f) for l in range(num_qs)], comments='!', dtype=int)
    # Depth distribution fractions into each node. Skipping the first (node count) column
    vqdist = np.loadtxt([next(f) for l in range(num_qs)])[:,1:]

    num_times = int(next(f))

    # Initialize storage arrays
    times = np.zeros(num_times)
    qs = np.zeros((num_times, num_qs))
    # State variables in the order they are present in the file. These are also going
    # to be the NetCDF variable names
    statevars = ('discharge', 'temp', 'salt', 'tss',  'alg1', 'alg2', 'alg3', 'zoo1',
                              'zoo2', 'ldoc', 'rdoc', 'lpoc', 'rpoc', 'nh4',  'no32',
                              'urea', 'ldon', 'rdon', 'lpon', 'rpon', 'po4',  'ldop',
                              'rdop', 'lpop', 'rpop', 'pip',  'cod',  'doxg', 'psi',
                              'dsi',  'alg1p','alg2p','alg3p','dic',  'talk')
    statedata = {}
    for v in statevars:
        statedata[v] = np.zeros((num_times, num_qs))

    for t in range(num_times):
        times[t] = float(next(f))
        for v in statevars:
            statedata[v][t,:] = np.loadtxt([next(f)])

print("Times:", times.shape)
print("Nodes:", nodes.shape)
print("VQdist:", vqdist.shape)
print("NO32:", statedata['no32'].shape)
statedata['no32']

Times: (365,)
Nodes: (193,)
VQdist: (193, 10)
NO32: (365, 193)


array([[ 0.1161,  0.1161,  0.712 , ..., 32.5   ,  6.852 ,  6.148 ],
       [ 0.1221,  0.1221,  0.7112, ..., 32.5   ,  6.852 ,  6.148 ],
       [ 0.1248,  0.1248,  0.7031, ..., 32.5   ,  6.852 ,  6.148 ],
       ...,
       [ 0.1217,  0.1217,  0.7224, ..., 27.5   ,  5.711 ,  5.033 ],
       [ 0.1209,  0.1209,  0.7221, ..., 27.5   ,  5.711 ,  5.033 ],
       [ 0.1191,  0.1191,  0.7205, ..., 27.5   ,  5.711 ,  5.033 ]])

In [3]:
cdf = Dataset(out_file, "w")

time_dim = cdf.createDimension("time", len(times))
node_dim = cdf.createDimension("node", num_qs)
siglay_dim = cdf.createDimension("siglay", vqdist.shape[1])

time_var = cdf.createVariable("time", "f4", ("time",))
time_var.unit = "hours"
cdf['time'][:] = times
node_var = cdf.createVariable("node", "i4", ("node",))
cdf['node'][:] = nodes
vqdist_var = cdf.createVariable("vqdist", "f4", ("node","siglay"))
cdf['vqdist'][:] = vqdist

for v in statevars:
    var = cdf.createVariable(v, "f4", ("time","node"))
    cdf[v][:] = statedata[v]

cdf

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): time(365), node(193), siglay(10)
    variables(dimensions): float32 time(time), int32 node(node), float32 vqdist(node, siglay), float32 discharge(time, node), float32 temp(time, node), float32 salt(time, node), float32 tss(time, node), float32 alg1(time, node), float32 alg2(time, node), float32 alg3(time, node), float32 zoo1(time, node), float32 zoo2(time, node), float32 ldoc(time, node), float32 rdoc(time, node), float32 lpoc(time, node), float32 rpoc(time, node), float32 nh4(time, node), float32 no32(time, node), float32 urea(time, node), float32 ldon(time, node), float32 rdon(time, node), float32 lpon(time, node), float32 rpon(time, node), float32 po4(time, node), float32 ldop(time, node), float32 rdop(time, node), float32 lpop(time, node), float32 rpop(time, node), float32 pip(time, node), float32 cod(time, node), float32 doxg(time, node), float32 psi(time, node), float32 ds

In [4]:
cdf.close()